# Xorbits 推理 (Xinference)

[Xinference](https://github.com/xorbitsai/inference) 是一个强大且通用的库，旨在为 LLM、语音识别模型和多模态模型提供服务，即使在你的笔记本电脑上也能运行。它支持多种与 GGML 兼容的模型，例如 chatglm、baichuan、whisper、vicuna、orca 等等。本教程演示了如何将 Xinference 与 LangChain 结合使用。

## 安装

通过 PyPI 安装 `Xinference`:

In [ ]:
%pip install --upgrade --quiet  "xinference[all]"

## 在本地或分布式集群中部署 Xinference。

本地部署，运行 `xinference`。

要在集群中部署 Xinference，首先使用 `xinference-supervisor` 启动一个 Xinference supervisor。您还可以使用 `-p` 参数指定端口，`-H` 参数指定主机。默认端口为 9997。

然后，在您希望运行 worker 的每个服务器上使用 `xinference-worker` 启动 Xinference workers。

您可以查阅 [Xinference](https://github.com/xorbitsai/inference) 的 README 文件以获取更多信息。
## Wrapper

要将 Xinference 与 LangChain 一起使用，您首先需要启动一个模型。您可以使用命令行界面 (CLI) 来执行此操作：

In [13]:
!xinference launch -n vicuna-v1.3 -f ggmlv3 -q q4_0

Model uid: 7167b2b0-2a04-11ee-83f0-d29396a3f064


现在您可以使用 Xinference 的 LangChain 了：模型 UID 将会返回供您使用。

In [14]:
from langchain_community.llms import Xinference

llm = Xinference(
    server_url="http://0.0.0.0:9997", model_uid="7167b2b0-2a04-11ee-83f0-d29396a3f064"
)

llm(
    prompt="Q: where can we visit in the capital of France? A:",
    generate_config={"max_tokens": 1024, "stream": True},
)

' You can visit the Eiffel Tower, Notre-Dame Cathedral, the Louvre Museum, and many other historical sites in Paris, the capital of France.'

### 与 LLMChain 集成

In [16]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

template = "Where can we visit in the capital of {country}?"

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

generated = llm_chain.run(country="France")
print(generated)


A: You can visit many places in Paris, such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, the Champs-Elysées, Montmartre, Sacré-Cœur, and the Palace of Versailles.


最后，当您不再需要该模型时，请终止它：

In [17]:
!xinference terminate --model-uid "7167b2b0-2a04-11ee-83f0-d29396a3f064"